## Exercise 3: 

This notebook demonstrates how to process data to build predictive machine learning models using a network dataset. This dataset is something that a Threat Hunter, Security Operations (SOC) analyst or a detection engineer will encounter in their day-to-day role. We'll use a custom built tool and **pandas** for data manipulation, **numpy** for making data become ML-ready

**What's the story?**

You are a threat hunter who is proactively looking to secure your organization. You create a hypothesis that you will find some sneaky malicious activity and start looking at network data. After the EDA process and further investigation by the Incident Response team (IR), you realize that there's benign and malicious network traffic. You are eager to catch such activity in the future and you set out on this mission! This is real world network data, fairly voluminous and not so kind to you. 


### Key Questions:
- What does my security spidey-sense have to say on distinguishing factors between malicious and benign?
- At what point do we want to identify it?

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import ipaddress
 
from utils.pcap import pcap_to_dataframe, extract_streams

# Load data
There are two ways to load the data:

- Directly reading a `pcap` and converting it to a Pandas `DataFrame`,
- Preloading the dataframe to a `.pkl` file. For more information on pickle files check the [article from RealPython about pickle module](https://realpython.com/python-pickle-module/).

In [20]:
malicious_pcap = "../data/mirai.pcap"
benign_pcap = "../data/benign.pcapng"
malicious_pkl_path = "../data/mirai.pkl"
benign_pkl_path = "../data/benign.pkl"
malicious_stream_path = "../data/mirai_stream.pkl"
benign_stream_path = "../data/benign_stream.pkl"

In [ ]:
# First time you run this, you should create your own pkl. Untrusted pickle files can cause deserialization harm creating remote code execution (RCE) opportunities. 
# For subsequent runs, set this flag to True
READ_FROM_PKL = False

In [ ]:
if READ_FROM_PKL:
    malicious_df = pd.read_pickle(malicious_pkl_path)
    benign_df = pd.read_pickle(benign_pkl_path)
    
    malicious_stream_df = pd.read_pickle("data/mirai_stream.pkl")
    benign_stream_df = pd.read_pickle("data/benign_stream.pkl")
else:
    malicious_df = pcap_to_dataframe(malicious_pcap)
    benign_df = pcap_to_dataframe(benign_pcap)
    malicious_stream_df = extract_streams(malicious_df)
    benign_stream_df = extract_streams(benign_df)
    # Save to pkl for accelerated processing in subsequent runs
    malicious_df.to_pickle(malicious_pkl_path)
    benign_df.to_pickle(benign_pkl_path)
    malicious_stream_df.to_pickle(malicious_stream_path)
    benign_stream_df.to_pickle(benign_stream_path)

In [10]:
malicious_df.sample(n=10)

,Timestamp,Source IP,Destination IP,Source Port,Destination Port,Payload,Packet Length,Protocol
248770,1540451434.032359,None,None,NaN,NaN,ARP who has 192.168.2.58 says 192.168.2.110 / ...,60,NaN
81167,1540449346.298863,192.168.2.108,122.248.234.207,32761.0,10240.0,Raw,48,17.0
47424,1540448124.304393,192.168.2.113,61.220.62.219,43517.0,80.0,,40,6.0
333901,1540451778.169512,None,None,NaN,NaN,ARP who has 192.168.2.158 says 192.168.2.110 /...,60,NaN
471288,1540452334.484407,None,None,NaN,NaN,ARP who has 192.168.2.187 says 192.168.2.110 /...,60,NaN
352686,1540451854.800959,192.168.2.108,50.19.254.134,32761.0,10240.0,Raw,48,17.0
65016,1540448760.607177,192.168.2.115,192.168.2.1,4067.0,53.0,DNS Qry b'xmpp.samsungsmartcam.com.Speedport_W...,83,17.0
503240,1540452464.389127,None,None,NaN,NaN,ARP who has 192.168.2.212 says 192.168.2.110 /...,60,NaN
683993,1540453196.220302,None,None,NaN,NaN,ARP who has 192.168.2.162 says 192.168.2.110 /...,60,NaN
149966,1540451010.887448,None,None,NaN,NaN,ARP who has 192.168.2.249 says 192.168.2.110 /...,60,NaN


In [11]:
malicious_df.shape

(764137, 8)

In [12]:
# Copy the dataframes to a features dataframe while omitting the packets with incomplete information such as NaN src/dst ips/ports
malicious_features = malicious_df.dropna(subset=["Source IP", "Destination IP", "Source Port", "Destination Port"])
benign_features = benign_df.dropna(subset=["Source IP", "Destination IP", "Source Port", "Destination Port"])

In [13]:
malicious_features.sample(n=10)

,Timestamp,Source IP,Destination IP,Source Port,Destination Port,Payload,Packet Length,Protocol
649677,1540453057.965771,192.168.2.115,192.168.2.1,3218.0,53.0,DNS Qry b'xmpp.samsungsmartcam.com.',50,17.0
9256,1540446706.299648,192.168.2.108,122.248.234.207,32761.0,10240.0,Raw,48,17.0
468608,1540452324.062125,192.168.2.110,192.168.2.107,21897.0,23.0,Padding,26,6.0
98581,1540449992.092292,192.168.2.115,192.168.2.1,4945.0,53.0,DNS Qry b'xmpp.samsungsmartcam.com.',50,17.0
16825,1540446986.88477,192.168.2.115,192.168.2.1,2901.0,53.0,DNS Qry b'time.nist.gov.Speedport_W_724V_01011...,72,17.0
62663,1540448676.752987,192.168.2.104,192.168.2.1,1525.0,53.0,DNS Qry b'10024main.broadlink.com.cn.',52,17.0
37526,1540447755.396392,192.168.2.108,52.25.66.250,33255.0,8280.0,Padding,26,6.0
89193,1540449643.751238,192.168.2.113,120.24.59.150,50861.0,10240.0,Raw,48,17.0
410273,1540452087.643424,192.168.2.107,192.168.2.110,23.0,21897.0,Padding,26,6.0
718091,1540453333.456501,192.168.2.113,50.19.254.134,54795.0,21047.0,,40,6.0


In [14]:
malicious_features.shape

(154090, 8)

#### How much has the dataset reduced after handling incomplete/real-world data?

# Numerical features
Post processing numbers to ... better numbers that describe context or add more data/information.

## Cumulative

Summarize your numerical features and give them a new meaning and utility.

In [15]:
malicious_features["src_ip_total_bytes"] = malicious_features.groupby("Source IP")[
    "Packet Length"
].cumsum()

/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/1408483151.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  malicious_features["src_ip_total_bytes"] = malicious_features.groupby("Source IP")[


In [16]:
benign_features["src_ip_total_bytes"] = benign_features.groupby("Source IP")[
    "Packet Length"
].cumsum()

/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/1306332885.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benign_features["src_ip_total_bytes"] = benign_features.groupby("Source IP")[


In [17]:
malicious_features["dst_ip_total_bytes"] = malicious_features.groupby("Destination IP")[
    "Packet Length"
].cumsum()

/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/3369579269.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  malicious_features["dst_ip_total_bytes"] = malicious_features.groupby("Destination IP")[


In [18]:
benign_features["dst_ip_total_bytes"] = benign_features.groupby("Destination IP")[
    "Packet Length"
].cumsum()

/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/2323472700.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benign_features["dst_ip_total_bytes"] = benign_features.groupby("Destination IP")[


In [19]:
malicious_features.sample(n=10)

,Timestamp,Source IP,Destination IP,Source Port,Destination Port,Payload,Packet Length,Protocol,src_ip_total_bytes,dst_ip_total_bytes
495012,1540452430.890582,192.168.2.113,61.220.62.219,45641.0,80.0,,40,6.0,1329571,35800
532928,1540452584.805687,192.168.2.108,61.188.37.216,32761.0,10240.0,Raw,48,17.0,1915986,551688
84354,1540449464.106211,192.168.2.113,61.220.62.219,44177.0,80.0,,40,6.0,679362,17800
100798,1540450077.435515,192.168.2.104,192.168.2.1,1696.0,53.0,DNS Qry b'10024backup.broadlink.com.cn.',54,17.0,120006,711878
223167,1540451330.713841,192.168.2.103,192.168.2.110,23.0,21897.0,Padding,26,6.0,7502,42265
90867,1540449705.612388,192.168.2.115,192.168.2.1,4761.0,53.0,DNS Qry b'time.nist.gov.Speedport_W_724V_01011...,72,17.0,411727,640358
143817,1540450971.536512,192.168.2.107,192.168.2.110,57206.0,23.0,,32,6.0,7010,12890
54820,1540448396.706595,192.168.2.113,120.24.59.150,50861.0,10240.0,Raw,48,17.0,446201,48952
549660,1540452652.525887,192.168.2.104,192.168.2.1,2014.0,53.0,DNS Qry b'10024main.broadlink.com.cn.',52,17.0,203926,1239629
495941,1540452434.974706,192.168.2.108,52.25.66.250,41318.0,8280.0,Padding,26,6.0,1869940,137644


## Numerical conversions

Convert numerical features to usable numbers.

In [21]:
def ip_to_numeric(ip):
    try:
        ip_obj = ipaddress.ip_interface(ip)
        ip = int(ip_obj.network.network_address)
    except ValueError:
        ip = 0

    return ip

In [22]:
malicious_features["Numeric Source IP"] = malicious_features["Source IP"].apply(
    ip_to_numeric
)

malicious_features["Numeric Destination IP"] = malicious_features["Destination IP"].apply(
    ip_to_numeric
)

/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/2803342481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  malicious_features["Numeric Source IP"] = malicious_features["Source IP"].apply(
/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/2803342481.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  malicious_features["Numeric Destination IP"] = malicious_features["Destination IP"].apply(


In [23]:
benign_features["Numeric Source IP"] = benign_features["Source IP"].apply(
    ip_to_numeric
)

benign_features["Numeric Destination IP"] = benign_features["Destination IP"].apply(
    ip_to_numeric
)

/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/3860816201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benign_features["Numeric Source IP"] = benign_features["Source IP"].apply(
/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/3860816201.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benign_features["Numeric Destination IP"] = benign_features["Destination IP"].apply(


In [24]:
# remove non-numeric IPs
malicious_features.pop("Source IP")
malicious_features.pop("Destination IP")

benign_features.pop("Source IP")
benign_features.pop("Destination IP")

4           224.0.0.2
5           224.0.0.2
6           224.0.0.2
7           224.0.0.2
26          224.0.0.2
             ...     
38585     194.247.5.1
38586    194.247.5.27
38587     194.247.5.1
38588    194.247.5.27
38589     194.247.5.1
Name: Destination IP, Length: 18615, dtype: object

# Categorical features
What about the text data? We can convert those to numbers too.

## Frequency encoding
Counts the population that corresponds to a specific category. The result is still a vector of categories, however not with 0s ans 1s, but with real numbers that indicate how often the category is encountered in the data.

In [25]:
malicious_frequency_encoding = (
   malicious_features["Destination Port"].value_counts(normalize=True).to_dict()
)

In [26]:
malicious_frequency_encoding

{10240.0: 0.40615224868583294,
 53.0: 0.21641897592316178,
 80.0: 0.11450451035109352,
 23.0: 0.06143163086507885,
 21897.0: 0.046453371406320984,
 8280.0: 0.04079434096956324,
 443.0: 0.019923421377117268,
 8000.0: 0.019274449996755143,
 21047.0: 0.010643130637938867,
 8080.0: 0.01025374780972159,
 1900.0: 0.0068271789214095656,
 32100.0: 0.004536309948731261,
 50364.0: 0.0024076838211434877,
 2323.0: 0.002018300992926212,
 57206.0: 0.0018430787202284378,
 51009.0: 0.0015575313128691024,
 123.0: 0.0012330456226880395,
 46734.0: 0.000947498215328704,
 68.0: 0.0008826010772924914,
 52777.0: 0.000623012525147641,
 41313.0: 0.0002920371211629567,
 9000.0: 0.0002660782659484717,
 67.0: 0.00017522227269777403,
 5353.0: 0.0001622428450905315,
 8629.0: 0.000149263417483289,
 138.0: 0.00011681484846518268,
 137.0: 9.73457070543189e-05,
 5355.0: 5.1917710428970085e-05,
 41291.0: 4.542799662534882e-05,
 41282.0: 3.8938282821727564e-05,
 41274.0: 3.8938282821727564e-05,
 3018.0: 1.946914141086378

In [27]:
benign_frequency_encoding = (
   benign_features["Destination Port"].value_counts(normalize=True).to_dict()
)

In [28]:
malicious_features["dst_port_freq_encoded"] = malicious_features["Destination Port"].map(
    malicious_frequency_encoding
)

/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/4292129847.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  malicious_features["dst_port_freq_encoded"] = malicious_features["Destination Port"].map(


In [29]:
benign_features["dst_port_freq_encoded"] = benign_features["Destination Port"].map(
    benign_frequency_encoding
)

/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/655699276.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benign_features["dst_port_freq_encoded"] = benign_features["Destination Port"].map(


## Derived Features

In [30]:
# Define a function to convert Scapy timestamps to pandas datetime
def scapy_timestamp_to_datetime(ts):
    return pd.to_datetime(
        ts.to_eng_string(), unit="s"
    )  # Convert to a format pandas understands


# Convert the Scapy timestamps to pandas datetime
malicious_features["Timestamp"] = malicious_features["Timestamp"].apply(scapy_timestamp_to_datetime).astype(int) / 10**9
benign_features["Timestamp"] = benign_features["Timestamp"].apply(scapy_timestamp_to_datetime).astype(int) / 10**9

/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/851821394.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  return pd.to_datetime(
/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/851821394.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  malicious_features["Timestamp"] = malicious_features["Timestamp"].apply(scapy_timestamp_to_datetime).astype(int) / 10**9
/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/851821394.py:3: FutureWarning: The b

In [31]:
malicious_features["Interarrival"] = malicious_features["Timestamp"].diff()
benign_features["Interarrival"] = benign_features["Timestamp"].diff()

/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/2373057836.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  malicious_features["Interarrival"] = malicious_features["Timestamp"].diff()
/var/folders/p3/_6p7wv_97gz2b8rl3hq40cmr0000gn/T/ipykernel_63378/2373057836.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benign_features["Interarrival"] = benign_features["Timestamp"].diff()


In [32]:
malicious_features.sample(n=5)

,Timestamp,Source Port,Destination Port,Payload,Packet Length,Protocol,src_ip_total_bytes,dst_ip_total_bytes,Numeric Source IP,Numeric Destination IP,dst_port_freq_encoded,Interarrival
351857,1.540452e+09,23.0,21897.0,Padding,26,6.0,348844,81409,3232236033,3232236142,0.046453,4.808903e-04
584190,1.540453e+09,21897.0,23.0,Padding,26,6.0,450695,1268467,3232236142,3232236033,0.061432,1.127720e-04
71148,1.540449e+09,55568.0,80.0,,40,6.0,573162,11280,3232236145,3410317659,0.114505,1.249313e-04
56837,1.540448e+09,50861.0,10240.0,Raw,48,17.0,462497,187048,3232236145,1035740632,0.406152,9.536743e-07
434966,1.540452e+09,21897.0,23.0,Padding,26,6.0,368711,41534,3232236142,3232236135,0.061432,2.145767e-06


In [33]:
malicious_features.dtypes

Timestamp                 float64
Source Port               float64
Destination Port          float64
Payload                    object
Packet Length               int64
Protocol                  float64
src_ip_total_bytes          int64
dst_ip_total_bytes          int64
Numeric Source IP           int64
Numeric Destination IP      int64
dst_port_freq_encoded     float64
Interarrival              float64
dtype: object

In [35]:
malicious_features.to_pickle("../data/malicious_features_numeric.pkl")
benign_features.to_pickle("../data/benign_features_numeric.pkl")

## One hot encoding
Binary encoding that creates a vector with 0s and 1s that correspond to specific categories. If your data had the category populated, mark it as 1 otherwise mark as 0. 

In [36]:
network_protocols = {
    1: "ICMP",
    6: "TCP",
    17: "UDP",
    23: "Telnet",
    41: "IPv6_encapsulation",
    47: "GRE",
    50: "ESP",
    51: "AH",
    53: "DNS",
    58: "ICMPv6",
    89: "OSPF",
    132: "SCTP",
    135: "SCTP",
    136: "UDPLite",
    137: "NETBIOS-NS",
    138: "NETBIOS-DGM",
    139: "NETBIOS-SSN",
    143: "IMAP",
    161: "SNMP",
    162: "SNMP_trap",
    443: "HTTPS",
    514: "Syslog",
    636: "LDAPS",
    989: "FTPS",
    993: "IMAPS",
    995: "POP3S",
    1080: "SOCKS_proxy",
    # Add more protocols as needed
}

In [37]:
def one_hot_port(port, df):
    new_df = pd.DataFrame()
    for protocol_port, protocol_name in network_protocols.items():
        new_df[protocol_name] = df[port].apply(
            lambda port: 1 if port == protocol_port else 0
        )
    return new_df

In [38]:
malicious_protocol_one_hot = one_hot_port("Destination Port", malicious_features)
malicious_features = pd.concat([malicious_features, malicious_protocol_one_hot], axis=1)

In [39]:
benign_protocol_one_hot = one_hot_port("Destination Port", benign_features)
benign_features = pd.concat([benign_features, benign_protocol_one_hot], axis=1)

In [40]:
malicious_features.to_pickle("../data/malicious_features.pkl")
benign_features.to_pickle("../data/benign_features.pkl")

In [43]:
malicious_features.describe()

,Timestamp,Source Port,Destination Port,Packet Length,Protocol,src_ip_total_bytes,dst_ip_total_bytes,Numeric Source IP,Numeric Destination IP,dst_port_freq_encoded,...,IMAP,SNMP,SNMP_trap,HTTPS,Syslog,LDAPS,FTPS,IMAPS,POP3S,SOCKS_proxy
count,1.540900e+05,154090.000000,154090.000000,154090.000000,154090.000000,1.540900e+05,1.540900e+05,1.540900e+05,1.540900e+05,154090.000000,...,154090.0,154090.0,154090.0,154090.000000,154090.0,154090.0,154090.0,154090.0,154090.0,154090.000000
mean,1.540450e+09,31461.554105,6758.415673,45.299890,13.348420,6.480438e+05,2.883691e+05,3.231670e+09,1.970533e+09,0.233578,...,0.0,0.0,0.0,0.019923,0.0,0.0,0.0,0.0,0.0,0.000006
std,2.080248e+03,18801.836599,7691.364297,17.558657,5.180108,5.928930e+05,3.112853e+05,4.278200e+07,1.039605e+09,0.156511,...,0.0,0.0,0.0,0.139738,0.0,0.0,0.0,0.0,0.0,0.002547
min,1.540446e+09,0.000000,23.000000,20.000000,6.000000,2.600000e+01,2.600000e+01,0.000000e+00,1.347441e+08,0.000006,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
25%,1.540448e+09,21897.000000,53.000000,40.000000,6.000000,1.617865e+05,6.316800e+04,3.232236e+09,8.740708e+08,0.061432,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
50%,1.540450e+09,32761.000000,8280.000000,48.000000,17.000000,4.427145e+05,1.624365e+05,3.232236e+09,2.014854e+09,0.216419,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
75%,1.540452e+09,50861.000000,10240.000000,48.000000,17.000000,1.050067e+06,4.276460e+05,3.232236e+09,3.232236e+09,0.406152,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
max,1.540454e+09,65267.000000,65267.000000,166.000000,17.000000,2.207294e+06,1.423395e+06,3.232236e+09,4.294967e+09,0.406152,...,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,1.000000


In [42]:
benign_features.describe()

,Timestamp,Source Port,Destination Port,Packet Length,Protocol,src_ip_total_bytes,dst_ip_total_bytes,Numeric Source IP,Numeric Destination IP,dst_port_freq_encoded,...,IMAP,SNMP,SNMP_trap,HTTPS,Syslog,LDAPS,FTPS,IMAPS,POP3S,SOCKS_proxy
count,1.861500e+04,18615.000000,18615.000000,18615.000000,18615.000000,1.861500e+04,1.861500e+04,1.861500e+04,1.861500e+04,18615.000000,...,18615.0,18615.000000,18615.000000,18615.000000,18615.000000,18615.0,18615.0,18615.0,18615.0,18615.0
mean,1.561492e+09,26100.062208,12621.709911,297.704110,15.524899,4.799977e+05,4.657947e+05,2.673749e+09,2.754457e+09,0.078901,...,0.0,0.000107,0.008488,0.008434,0.008971,0.0,0.0,0.0,0.0,0.0
std,5.667702e+07,24859.356286,17691.465555,447.200549,6.462379,8.002136e+05,7.824794e+05,1.059710e+09,1.059214e+09,0.075002,...,0.0,0.010365,0.091740,0.091451,0.094293,0.0,0.0,0.0,0.0,0.0
min,1.355254e+09,7.000000,7.000000,9.000000,6.000000,2.000000e+01,2.000000e+01,0.000000e+00,1.684301e+07,0.000054,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
25%,1.509911e+09,5690.000000,1303.000000,56.000000,17.000000,5.966000e+03,6.962000e+03,1.418889e+09,1.418889e+09,0.009079,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
50%,1.574093e+09,15020.000000,5690.000000,180.000000,17.000000,1.169920e+05,1.022400e+05,3.232237e+09,3.232237e+09,0.049530,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
75%,1.597832e+09,54505.500000,15020.000000,180.000000,17.000000,4.722740e+05,4.641315e+05,3.239633e+09,3.628225e+09,0.172710,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
max,1.671036e+09,65535.000000,65535.000000,8780.000000,47.000000,2.760212e+06,2.732799e+06,3.640661e+09,4.294967e+09,0.180929,...,0.0,1.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0
